In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.utils.multiclass import unique_labels
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold

In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Read Files

In [3]:
# normalized data, with GA effect
#data = pd.read_csv("esetSC2_normalized.csv")

# data here is normalized, and the GA effect is removed.
data = pd.read_csv("esetSC2_remove_effect.csv")

# data information
data_info = pd.read_csv('anoSC2_v20_nokey.csv')

# GA information
GA_info = pd.read_csv("allSamplesSC1and2.csv")

GA_infomation = GA_info.set_index("SampleID")
GA_infomation = GA_infomation.loc[data.columns]
data_infomation = data_info.reset_index().set_index("SampleID").drop(columns = 'index')
data_infomation = data_infomation.loc[data.columns]
data_infomation = data_infomation[(data_infomation['Group']=='Control') | (data_infomation['Group']=='sPTD')]
data = data[data_infomation.index]
data_copy = data

## Split the data into train and test

In [4]:
def SampleList_To_Group(sample_list, data_infomation = data_infomation):
    control_sample = []
    sPTD_sample = []
    sample_group = []
    for sample in sample_list:
        group = data_infomation.loc[[sample]][['Group']].values[0][0]
        if group == 'Control':
            control_sample.append(sample)
            sample_group.append('Control')
        if group == 'sPTD':
            sPTD_sample.append(sample)
            sample_group.append('sPTD')
    return sample_group, control_sample, sPTD_sample

In [5]:
sample_list = list(data.columns)
sample_group, control_sample, sPTD_sample = SampleList_To_Group(sample_list)
xtrain,xtest,ytrain,ytest = train_test_split(data.transpose(), sample_group,test_size=0.2, random_state=1)

## Preliminary Feature Selection: Choose genes with high t-test score in different GA and all GAs

In [6]:
control_sample = []
sPTD_sample = []
sapmles = list(xtrain.index)
sample_group, control_sample, sPTD_sample = SampleList_To_Group(sapmles)
sPTD_data = xtrain.transpose()[control_sample + sPTD_sample]
df = sPTD_data
df[['Control_mean', 'Control_std']] = df[control_sample].agg(['mean', 'std'], axis=1)
df[['sPTD_mean', 'sPTD_std']] = df[sPTD_sample].agg(['mean', 'std'], axis=1)
def welch_t_test(row):
    return (
        (row['Control_mean'] - row['sPTD_mean']) / 
        np.sqrt(row['Control_std']/len(control_sample) + row['sPTD_std']/len(sPTD_sample))
    )
df['similarity'] = df[['Control_mean', 'Control_std', 'sPTD_mean', 'sPTD_std']].apply(welch_t_test, axis=1)
df['similarity_abs'] = abs(df['similarity'])
df_sorted = df.sort_values('similarity_abs',ascending=False)
data_infomation_train = data_infomation.loc[xtrain.index]
GA_values = np.unique(data_infomation_train.loc[df_sorted.columns[:-5]][['GA']].values)
GA_values = [x for x in GA_values if str(x) != 'nan']
df_dict_GA = {}
for key in GA_values:
    GA_samples = data_infomation_train.loc[data_infomation_train['GA']==key].index
    df_dict_GA[key] = GA_samples.tolist()   
df  = df_sorted.drop(columns=['Control_mean', 'Control_std', 'sPTD_mean', 'sPTD_std', 'similarity','similarity_abs'])
for key in df_dict_GA:
    df_dict_GA[key] = df[df_dict_GA[key]]
df_dict_GA=pickle.load(open('df_dict_GA_1.txt', 'rb'))

In [7]:
gene_similarity = []
for key in df_dict_GA:
    df = df_dict_GA[key]
    gene = df.loc[df["similarity_abs"] >= 3].index.tolist()
    gene_similarity = gene_similarity + gene
gene = sPTD_data.loc[df_sorted["similarity_abs"] >= 2].index.tolist()
gene_similarity = gene_similarity + gene
gene_similarity = np.unique(gene_similarity)
xtrain = xtrain[gene_similarity]
xtest = xtest[gene_similarity]

## Feature Selection: Recursive Feature Elimination

In [8]:
# Cross Validation
def Cross_Validation(df = xtrain):
    kf = KFold(n_splits=5)
    train_set_list = []
    validation_set_list = []
    for train_set, validation_set in kf.split(df):
        train_set = df.iloc[train_set]
        validation_set = df.iloc[validation_set]
        train_set_list.append(train_set)
        validation_set_list.append(validation_set)
    return train_set_list, validation_set_list

### Linear SVC

In [9]:
from sklearn.svm import LinearSVC
def prediction_knn(features, df = xtrain, y = ytrain):
    df = df[features]
    total_error_ = []
    train_set_list, validation_set_list = Cross_Validation(df)
    clf = KNeighborsClassifier(n_neighbors=1)
    for i in range(len(train_set_list)):
        train = train_set_list[i]
        test = validation_set_list[i]
        sample_group, control_sample, sPTD_sample = SampleList_To_Group(list(train.index))
        y_train = sample_group
        sample_group, control_sample, sPTD_sample = SampleList_To_Group(list(test.index))
        y_test = sample_group
        clf.fit(train, y_train)
        pred = clf.predict(test).tolist()
        error = 0
        for i in range(len(pred)):
            if y_test[i] != pred[i]:
                error += 1
        total_error_.append(error)
    total_error = np.sum(total_error_)
    sample_number = df.shape[0]
    error_rate =  total_error/sample_number
    return error_rate

In [10]:
rfe_lib = {}
feature_lib = {}
feature_number = xtrain.shape[0]
df = xtrain
for n_features_to_select in range(1,feature_number):
    clf = SVC(kernel="linear", C=1)
    rfe = RFE(estimator=clf, n_features_to_select=n_features_to_select, step=1)
    rfe.fit(df, ytrain)
    Features = df.columns[rfe.support_]
    error_rate = prediction_knn(features = Features)
    rfe_lib[n_features_to_select] = error_rate
    feature_lib[n_features_to_select] = Features
p1 = list(rfe_lib.keys())
best_genes_numbers = p1.index(min(p1))+1
genes = list(feature_lib[best_genes_numbers])
clf = KNeighborsClassifier(n_neighbors=1)
xtrain_FS = xtrain[genes]
xtest_FS = xtest[genes]
clf.fit(xtrain_FS, ytrain)
knn_predict = clf.predict(xtest_FS)
prediction = list(knn_predict)
result = ytest
error_num = 0
for i in range(len(prediction)):
    if prediction[i] == result[i]:
        pass
    else:
        error_num = error_num+1
positive_number = 0        
negative_number = 0
TP_number = 0
TN_number = 0
for i in range(len(prediction)):
    if result[i] == 'Control':
        positive_number = positive_number+1
        if prediction[i] == result[i]:
            TP_number = TP_number+1
    if result[i] == 'sPTD':
        negative_number = negative_number+1
        if prediction[i] == result[i]:
            TN_number = TN_number+1
TP_rate = TP_number/positive_number
TN_rate = TN_number/negative_number
print(TP_rate)
print(TN_rate)
print(error_num)

0.9148936170212766
0.0
8


### Extra Trees Classifier

In [11]:
from sklearn.ensemble import ExtraTreesClassifier
def prediction_knn(features, df = xtrain, y = ytrain):
    df = df[features]
    total_error_ = []
    train_set_list, validation_set_list = Cross_Validation(df)
    clf = KNeighborsClassifier(n_neighbors=1)
    for i in range(len(train_set_list)):
        train = train_set_list[i]
        test = validation_set_list[i]
        sample_group, control_sample, sPTD_sample = SampleList_To_Group(list(train.index))
        y_train = sample_group
        sample_group, control_sample, sPTD_sample = SampleList_To_Group(list(test.index))
        y_test = sample_group
        clf.fit(train, y_train)
        pred = clf.predict(test).tolist()
        error = 0
        for i in range(len(pred)):
            if y_test[i] != pred[i]:
                error += 1
        total_error_.append(error)
    total_error = np.sum(total_error_)
    sample_number = df.shape[0]
    error_rate =  total_error/sample_number
    return error_rate

In [12]:
rfe_lib = {}
feature_lib = {}
feature_number = xtrain.shape[0]
df = xtrain
for n_features_to_select in range(1,feature_number):
    clf = ExtraTreesClassifier(n_estimators=100)
    rfe = RFE(estimator=clf, n_features_to_select=n_features_to_select, step=1)
    rfe.fit(df, ytrain)
    Features = df.columns[rfe.support_]
    error_rate = prediction_knn(features = Features)
    rfe_lib[n_features_to_select] = error_rate
    feature_lib[n_features_to_select] = Features
p1 = list(rfe_lib.keys())
best_genes_numbers = p1.index(min(p1))+1
genes = list(feature_lib[best_genes_numbers])
clf = KNeighborsClassifier(n_neighbors=1)
xtrain_FS = xtrain[genes]
xtest_FS = xtest[genes]
clf.fit(xtrain_FS, ytrain)
knn_predict = clf.predict(xtest_FS)
prediction = list(knn_predict)
result = ytest
error_num = 0
for i in range(len(prediction)):
    if prediction[i] == result[i]:
        pass
    else:
        error_num = error_num+1
positive_number = 0        
negative_number = 0
TP_number = 0
TN_number = 0
for i in range(len(prediction)):
    if result[i] == 'Control':
        positive_number = positive_number+1
        if prediction[i] == result[i]:
            TP_number = TP_number+1
    if result[i] == 'sPTD':
        negative_number = negative_number+1
        if prediction[i] == result[i]:
            TN_number = TN_number+1
TP_rate = TP_number/positive_number
TN_rate = TN_number/negative_number
print(TP_rate)
print(TN_rate)
print(error_num)

0.9148936170212766
0.5
6


### L1-based

In [13]:
from sklearn.svm import LinearSVC
def prediction_knn(features, df = xtrain, y = ytrain):
    df = df[features]
    total_error_ = []
    train_set_list, validation_set_list = Cross_Validation(df)
    clf = KNeighborsClassifier(n_neighbors=1)
    for i in range(len(train_set_list)):
        train = train_set_list[i]
        test = validation_set_list[i]
        sample_group, control_sample, sPTD_sample = SampleList_To_Group(list(train.index))
        y_train = sample_group
        sample_group, control_sample, sPTD_sample = SampleList_To_Group(list(test.index))
        y_test = sample_group
        clf.fit(train, y_train)
        pred = clf.predict(test).tolist()
        error = 0
        for i in range(len(pred)):
            if y_test[i] != pred[i]:
                error += 1
        total_error_.append(error)
    total_error = np.sum(total_error_)
    sample_number = df.shape[0]
    error_rate =  total_error/sample_number
    return error_rate

In [14]:
rfe_lib = {}
feature_lib = {}
feature_number = xtrain.shape[0]
df = xtrain
for n_features_to_select in range(1,feature_number):
    clf = LinearSVC(C=0.01, penalty="l1", dual=False)
    rfe = RFE(estimator=clf, n_features_to_select=n_features_to_select, step=1)
    rfe.fit(df, ytrain)
    Features = df.columns[rfe.support_]
    error_rate = prediction_knn(features = Features)
    rfe_lib[n_features_to_select] = error_rate
    feature_lib[n_features_to_select] = Features
p1 = list(rfe_lib.keys())
best_genes_numbers = p1.index(min(p1))+1
genes = list(feature_lib[best_genes_numbers])
clf = KNeighborsClassifier(n_neighbors=1)
xtrain_FS = xtrain[genes]
xtest_FS = xtest[genes]
clf.fit(xtrain_FS, ytrain)
knn_predict = clf.predict(xtest_FS)
prediction = list(knn_predict)
result = ytest
error_num = 0
for i in range(len(prediction)):
    if prediction[i] == result[i]:
        pass
    else:
        error_num = error_num+1
positive_number = 0        
negative_number = 0
TP_number = 0
TN_number = 0
for i in range(len(prediction)):
    if result[i] == 'Control':
        positive_number = positive_number+1
        if prediction[i] == result[i]:
            TP_number = TP_number+1
    if result[i] == 'sPTD':
        negative_number = negative_number+1
        if prediction[i] == result[i]:
            TN_number = TN_number+1
TP_rate = TP_number/positive_number
TN_rate = TN_number/negative_number
print(TP_rate)
print(TN_rate)
print(error_num)

0.9361702127659575
0.25
6
